In [1]:
!pip install transformers
!pip install torch

In [2]:
!pip install openai-whisper


In [3]:
!pip install ffmpeg

In [4]:
!pip install ffmpeg-python

In [5]:
import pandas as pd
df=pd.read_csv('transcripts.csv.csv')


C:\Users\User\AppData\Local\Temp\ipykernel_9752\2432090198.py:2: DtypeWarning: Columns (1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('transcripts.csv.csv')


In [6]:
df.head

<bound method NDFrame.head of                                                  text  \
0        he had to spit some tobacco out of his mouth   
1        he had to spit some tobacco out of his mouth   
2        he had to spit some tobacco out of his mouth   
3        he had to spit some tobacco out of his mouth   
4        he had to spit some tobacco out of his mouth   
...                                               ...   
145102                              it's on the level   
145103        who do you feel is responsible for that   
145104  you'll be the only one who'll swipe any of it   
145105                      nothing's going to happen   
145106                           it belongs to me now   

                                          Unnamed: 1  \
0       he had to spit some tobacco out of his mouth   
1       he had to spit some tobacco out of his mouth   
2       he had to spit some tobacco out of his mouth   
3       he had to spit some tobacco out of his mouth   
4    

In [7]:

text_array=df['text'].tolist()
text_array

['he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit some tobacco out of his mouth',
 'he had to spit som

In [8]:
df=pd.read_csv('train gloss.csv')

In [9]:
!pip install mediapipe

In [10]:
pip install mediapipe opencv-python

In [11]:
df=pd.read_csv('train gloss.csv')

In [12]:
gloss_array=df['gloss'].tolist()
gloss_array

['MEMBERSHIP PARLIAMENT SEE MINUTE\n',
 'APPROVAL MINUTE DESC-PREVIOUS SIT SEE MINUTE\n',
 'MEMBERSHIP PARLIAMENT SEE MINUTE\n',
 'VERIFICATION CREDENTIALS SEE MINUTE\n',
 'DOCUMENT RECEIVE SEE MINUTE\n',
 'WRITE STATEMENT AND DESC-ORAL QUESTION TABLE SEE MINUTE\n',
 'PETITION SEE MINUTE\n',
 'TEXT AGREEMENT DESC-FORWARD BY COUNCIL SEE MINUTE\n',
 'ACTION TAKE ON PARLIAMENT X-POSS RESOLUTION SEE MINUTE\n',
 'AGENDA FOR NEXT SIT SEE MINUTE\n',
 'CLOSURE SIT\n',
 'SIT BE DESC-CLOSE AT 7.45 DESC-P.M.\n',
 'ELECTION VICE PRESIDENT EUROPEAN PARLIAMENT DEADLINE FOR SUBMIT NOMINATION SEE MINUTE\n',
 'SIT BE SUSPEND AT 12.40 DESC-P.M. AND RESUME AT 3.00 DESC-P.M.\n',
 'ELECTION QUAESTOR EUROPEAN PARLIAMENT DEADLINE FOR SUBMIT NOMINATION SEE MINUTE\n',
 'SIT BE SUSPEND AT 3.25 DESC-P.M. AND RESUME AT 6.00 DESC-P.M.\n',
 'AGENDA FOR NEXT SIT SEE MINUTE\n',
 'CLOSURE SIT\n',
 'SIT BE DESC-CLOSE AT 6.15 DESC-P.M.\n',
 'OPEN SIT\n',
 'SIT BE DESC-OPEN AT 9.35 DESC-A.M.\n',
 'DOCUMENT RECEIVE SEE MI

In [13]:
import numpy as np
import cv2

# Define predefined gesture templates for common glosses (expanded with more keypoints)
gesture_templates = {
    "HELLO": [
        [[0.4, 0.5], [0.35, 0.55], [0.45, 0.6], [0.5, 0.45], [0.55, 0.48]],  # Expanded gesture
        [[0.42, 0.52], [0.36, 0.57], [0.46, 0.63], [0.5, 0.47], [0.55, 0.5]],  # Second frame with more points
    ],
    "THANK YOU": [
        [[0.4, 0.5], [0.35, 0.55], [0.45, 0.6], [0.5, 0.45], [0.55, 0.48]],  # More points for the "thank you" gesture
        [[0.42, 0.52], [0.36, 0.57], [0.46, 0.63], [0.5, 0.47], [0.55, 0.5]],  # More points
    ],
    "YES": [
        [[0.4, 0.5], [0.35, 0.55], [0.45, 0.6], [0.5, 0.45], [0.55, 0.48]],  # Fist gesture with more points
        [[0.42, 0.52], [0.36, 0.57], [0.46, 0.63], [0.5, 0.47], [0.55, 0.5]],  # More points
    ],
    "NO": [
        [[0.3, 0.5], [0.25, 0.55], [0.35, 0.6], [0.4, 0.5], [0.45, 0.52]],  # Head shake gesture with more points
        [[0.32, 0.52], [0.27, 0.57], [0.37, 0.62], [0.42, 0.53], [0.47, 0.55]],  # More points
    ],
    "VERIFICATION": [
        [[0.4, 0.5], [0.35, 0.55], [0.45, 0.6], [0.5, 0.45], [0.55, 0.48]],  # Hand swipe with more points
        [[0.42, 0.52], [0.36, 0.57], [0.46, 0.63], [0.5, 0.47], [0.55, 0.5]],  # More points
    ]
}

# Function to map gloss to gesture keypoints
def map_gloss_to_gesture(gloss):
    """
    Automatically map gloss to predefined gesture template.
    """
    gloss = gloss.upper()  # Convert to uppercase for matching
    for gesture in gesture_templates:
        if gesture in gloss:
            return gesture_templates[gesture]
    
    # Default gesture if no match (can be customized)
    return [
        [[0.4, 0.5], [0.35, 0.55], [0.45, 0.6], [0.5, 0.45], [0.55, 0.48]],  # Default gesture (hand wave)
        [[0.42, 0.52], [0.36, 0.57], [0.46, 0.63], [0.5, 0.47], [0.55, 0.5]],  # Hand wave continued
    ]


# Function to visualize synthetic gesture
def visualize_synthetic_gesture(gloss_keypoints):
    # Create a blank frame to simulate gesture visualization
    frame = np.zeros((480, 640, 3), dtype=np.uint8)
    
    for keypoints in gloss_keypoints:
        frame[:] = 0  # Reset frame for each gesture pose
        
        # Convert the keypoints to (x, y) positions for visualization
        for i, (x, y) in enumerate(keypoints):
            x_pos, y_pos = int(x * 640), int(y * 480)  # Normalize to frame size
            cv2.circle(frame, (x_pos, y_pos), 10, (0, 255, 0), -1)  # Draw keypoint
        
        # Display the frame with the gesture pose
        cv2.imshow('Synthetic Gesture', frame)
        cv2.waitKey(200)  # Wait for 200ms between poses
        
    cv2.destroyAllWindows()

# Example: Generate gestures for glosses in a dataset

for gloss in gloss_array:
    print(f"Generating gesture for: {gloss}")
    gloss_keypoints = map_gloss_to_gesture(gloss)
    visualize_synthetic_gesture(gloss_keypoints)



Generating gesture for: MEMBERSHIP PARLIAMENT SEE MINUTE

Generating gesture for: APPROVAL MINUTE DESC-PREVIOUS SIT SEE MINUTE

Generating gesture for: MEMBERSHIP PARLIAMENT SEE MINUTE

Generating gesture for: VERIFICATION CREDENTIALS SEE MINUTE

Generating gesture for: DOCUMENT RECEIVE SEE MINUTE

Generating gesture for: WRITE STATEMENT AND DESC-ORAL QUESTION TABLE SEE MINUTE

Generating gesture for: PETITION SEE MINUTE

Generating gesture for: TEXT AGREEMENT DESC-FORWARD BY COUNCIL SEE MINUTE

Generating gesture for: ACTION TAKE ON PARLIAMENT X-POSS RESOLUTION SEE MINUTE

Generating gesture for: AGENDA FOR NEXT SIT SEE MINUTE

Generating gesture for: CLOSURE SIT

Generating gesture for: SIT BE DESC-CLOSE AT 7.45 DESC-P.M.

Generating gesture for: ELECTION VICE PRESIDENT EUROPEAN PARLIAMENT DEADLINE FOR SUBMIT NOMINATION SEE MINUTE

Generating gesture for: SIT BE SUSPEND AT 12.40 DESC-P.M. AND RESUME AT 3.00 DESC-P.M.

Generating gesture for: ELECTION QUAESTOR EUROPEAN PARLIAMENT DEAD

Generating gesture for: THAT CONCLUDE VOTE .

Generating gesture for: VOTE TIME CONTINUE

Generating gesture for: NEXT ITEM BE VOTE TIME .

Generating gesture for: CORRECTION TO VOTE AND VOTE INTENTION SEE MINUTE

Generating gesture for: SIT BE SUSPEND AT 2 DESC-P.M. AND RESUME AT 3 DESC-P.M.

Generating gesture for: APPROVAL MINUTE DESC-PREVIOUS SIT SEE MINUTE

Generating gesture for: AGENDA FOR NEXT PART SESSION SEE MINUTE

Generating gesture for: CLOSURE SIT

Generating gesture for: SIT DESC-CLOSE AT 23.15

Generating gesture for: OPEN SIT

Generating gesture for: SIT BE DESC-OPEN AT 10 DESC-A.M.

Generating gesture for: DOCUMENT RECEIVE SEE MINUTE

Generating gesture for: SHAREHOLDERSVOTE RIGHTS VOTE

Generating gesture for: BUDGET AID FOR DEVELOP COUNTRY VOTE

Generating gesture for: DESC-ILLEGAL FISHING VOTE

Generating gesture for: COUNTRY STRATEGY PAPERS MALAYSIUM , BRAZIL AND PAKISTAN VOTE

Generating gesture for: EC RUSSIUM DESC-SHORT STAY VISA AGREEMENT VOTE

Generating gest

Generating gesture for: FUTURE EUROPEAN UNION X-POSS DESC-OWN RESOURCE VOTE

Generating gesture for: FUTURE DESC-PROFESSIONAL FOOTBALL IN EUROPE VOTE

Generating gesture for: INTEGRATION DESC-NEW MEMBER STATE IN CAP VOTE

Generating gesture for: THAT CONCLUDE VOTE .

Generating gesture for: CORRECTION TO VOTE AND VOTE INTENTION SEE MINUTE

Generating gesture for: COMMUNICATION COUNCIL DESC-COMMON POSITION SEE MINUTE

Generating gesture for: MEMBERSHIP DESC-INTERPARLIAMENTARY DELEGATION SEE MINUTE

Generating gesture for: MEMBERSHIP PARLIAMENT SEE MINUTE

Generating gesture for: FORWARDING TEXT ADOPT DURING SIT SEE MINUTE

Generating gesture for: DATE FOR NEXT SIT SEE MINUTE

Generating gesture for: APPROVAL MINUTE DESC-PREVIOUS SIT SEE MINUTE

Generating gesture for: DOCUMENT RECEIVE SEE MINUTE

Generating gesture for: TEXT AGREEMENT DESC-FORWARD BY COUNCIL SEE MINUTE

Generating gesture for: DECISION CONCERN DESC-CERTAIN DOCUMENT SEE MINUTE

Generating gesture for: DESC-ORAL QUESTION 

Generating gesture for: COMPENSATION FOR DESC-CERTAIN FISHERY PRODUCT VOTE

Generating gesture for: FREITA REPORT

Generating gesture for: DESC-COMMON FISHERY POLICY DESC-REGIONAL DESC-ADVISORY COUNCIL VOTE

Generating gesture for: ATTWOOLL REPORT

Generating gesture for: SITUATION DESC-DISABLED WOMAN IN EUROPEAN UNION VOTE

Generating gesture for: HERRANZ GARCa REPORT

Generating gesture for: DESC-BASIC INFORMATION ON PURCHASE POWER PARITY VOTE

Generating gesture for: BOWLE REPORT

Generating gesture for: DESC-RAPID BORDER INTERVENTION TEAM VOTE

Generating gesture for: DEPREZ REPORT

Generating gesture for: CONSERVATION AND DESC-SUSTAINABLE EXPLOITATION FISHERY RESOURCE VOTE

Generating gesture for: GKLAVAKUS REPORT

Generating gesture for: GALILEO VOTE

Generating gesture for: MOTION FOR RESOLUTION

Generating gesture for: PUBLIC FINANCES IN EMU 2006 VOTE

Generating gesture for: LAUK REPORT

Generating gesture for: THAT CONCLUDE VOTING TIME .

Generating gesture for: CORRECTION TO

Generating gesture for: DESC-STRUCTURAL POLICY AND EU COHESION VOTE

Generating gesture for: REPORT PLEGUEZUELO AGUILAR

Generating gesture for: EU X-POSS AID FOR TRADE VOTE

Generating gesture for: REPORT DAVID MARTIN

Generating gesture for: DESC-DECENT WORK VOTE

Generating gesture for: REPORT PANAYOTOPOULO CASSIOTOU

Generating gesture for: THAT CONCLUDE VOTE .

Generating gesture for: VOTE TIME

Generating gesture for: NEXT ITEM BE VOTE .

Generating gesture for: FOR RESULT AND DESC-OR DETAIL ON VOTE SEE MINUTE

Generating gesture for: CORRECTION TO VOTE AND VOTE INTENTION SEE MINUTE

Generating gesture for: SIT BE SUSPEND AT 13.10 AND RESUME AT 15.00

Generating gesture for: APPROVAL MINUTE DESC-PREVIOUS SIT SEE MINUTE

Generating gesture for: AGENDA FOR NEXT SIT SEE MINUTE

Generating gesture for: CLOSURE SIT

Generating gesture for: SIT BE DESC-CLOSE AT 12 MIDNIGHT

Generating gesture for: DOCUMENT RECEIVE SEE MINUTE

Generating gesture for: TRANSFER APPROPRIATION SEE MINUTE

G

KeyboardInterrupt: 